In [1]:
import sys

import keras_tuner as kt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import callbacks
from tensorflow.keras import optimizers

from src.cross_validator import KerasTunerAllChannelsCrossValidator
from src.dataset import load_dataset_train_val_all_channels, \
    load_dataset_train_test_all_channels
from src.models import rnn_builder as bare_model_builder
from src.network_utils import count_params

PWD = '../../..'
sys.path.append(PWD)

tf.get_logger().setLevel('ERROR')

In [2]:
PROJECT_NAME = 'rnn'

OVERWRITE = False

DATASET_PATH = PWD + '/data/dataset/dataset.pkl'
TRIALS_DIR = PWD + f'/data/model_selection/channel_all/tuner'
CROSSVAL_DIR = PWD + f'/data/model_selection/channel_all/cross_val'

LR = 0.01
ES_MIN_DELTA = 0.01

N_EPOCHS = 500
BATCH_SIZE = 8096
MAX_TRIALS = 15
EXECUTIONS_PER_TRIAL = 2

TOP_N = 4
CROSSVAL_N_CV = 5
CROSSVAL_N_EXEC = 2
LOSS_WEIGHT = 1000

In [3]:
X_all, _, y_all, _ = load_dataset_train_test_all_channels(PWD)

X_all[(2, 11)].shape, y_all[(2, 11)].shape

((22134, 24), (22134,))

In [4]:
X_train, X_val, y_train, y_val = load_dataset_train_val_all_channels(PWD)

X_train = np.concatenate(list(X_train.values()))
y_train = np.concatenate(list(y_train.values()))
X_val = np.concatenate(list(X_val.values()))
y_val = np.concatenate(list(y_val.values()))

rng = np.random.default_rng(seed=42)

train_idx, val_idx = np.arange(len(X_train)), np.arange(len(X_val))
rng.shuffle(train_idx)
rng.shuffle(val_idx)

X_train, y_train = X_train[train_idx], y_train[train_idx]
X_val, y_val = X_val[val_idx], y_val[val_idx]

X_train.shape, y_train.shape, X_val.shape, y_val.shape

((84824, 24), (84824,), (21208, 24), (21208,))

# Model

In [5]:
def model_builder(hp: kt.HyperParameters) -> keras.Model:
    hp_rnn_type = hp.Choice("rnn_type", values=["lstm", "gru"])
    hp_n_neurons = hp.Choice("n_neurons", values=[16, 32, 64, 128, 256, 512])
    hp_n_hidden_layers = hp.Int("n_hidden_layers", min_value=0, max_value=1, default=0)
    hp_input_batch_normalization = hp.Boolean("input_batch_normalization")
    hp_normalize_signal = hp.Boolean("normalize_signal", default=False)

    model = bare_model_builder(hp_rnn_type, hp_n_neurons, hp_n_hidden_layers, hp_input_batch_normalization,
                               hp_normalize_signal)
    model.compile(loss='mse', optimizer=optimizers.Adam(LR), loss_weights=LOSS_WEIGHT)
    return model

In [6]:
model_builder(kt.HyperParameters()).summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 24, 1)             0         
                                                                 
 lstm (LSTM)                 (None, 16)                1152      
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
Total params: 1,169
Trainable params: 1,169
Non-trainable params: 0
_________________________________________________________________


In [7]:
model_callbacks = [
    callbacks.EarlyStopping(patience=60, min_delta=ES_MIN_DELTA),
    callbacks.ReduceLROnPlateau(monitor='loss', factor=0.9, patience=10)
]

# Bayesian tuner

In [8]:
bayesian_tuner = kt.BayesianOptimization(model_builder, objective='val_loss', executions_per_trial=EXECUTIONS_PER_TRIAL,
                                         max_trials=MAX_TRIALS, directory=TRIALS_DIR, project_name=PROJECT_NAME,
                                         overwrite=OVERWRITE)

bayesian_tuner.search(X_train, y_train, validation_data=[X_val, y_val], epochs=N_EPOCHS, callbacks=model_callbacks,
                      batch_size=BATCH_SIZE, verbose=3)

# Results

In [9]:
for i, hyperparameters in enumerate(bayesian_tuner.get_best_hyperparameters(TOP_N)):
    print(f'========== Model {i} ==========')
    print(hyperparameters.get_config()['values'])
    model_tmp = model_builder(hyperparameters)
    print('Number of parameters:', count_params(model_tmp))

========== Model 0 ==========
{'rnn_type': 'lstm', 'n_neurons': 16, 'n_hidden_layers': 1, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 3377
========== Model 1 ==========
{'rnn_type': 'gru', 'n_neurons': 64, 'n_hidden_layers': 1, 'input_batch_normalization': True, 'normalize_signal': True}
Number of parameters: 37985
========== Model 2 ==========
{'rnn_type': 'lstm', 'n_neurons': 16, 'n_hidden_layers': 0, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 1265
========== Model 3 ==========
{'rnn_type': 'gru', 'n_neurons': 16, 'n_hidden_layers': 0, 'input_batch_normalization': True, 'normalize_signal': True}
Number of parameters: 1025


# Cross-validation for top 5 models

In [10]:
cross_validator = KerasTunerAllChannelsCrossValidator(bayesian_tuner, list(X_all.values()), list(y_all.values()),
                                                      model_builder, directory=CROSSVAL_DIR, project_name=PROJECT_NAME,
                                                      n_epochs=N_EPOCHS, batch_size=BATCH_SIZE, n_top=TOP_N,
                                                      es_min_delta=ES_MIN_DELTA, n_cv=CROSSVAL_N_CV,
                                                      n_executions=CROSSVAL_N_EXEC, overwrite=OVERWRITE)
model_scores = cross_validator()

{'rnn_type': 'lstm', 'n_neurons': 16, 'n_hidden_layers': 1, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 3377
Got score: 8.7440 (8.8015, 8.6866)
Got score: 8.4785 (8.4335, 8.5235)
Got score: 8.5104 (8.4854, 8.5354)
Got score: 8.3959 (8.4004, 8.3914)
Got score: 8.7813 (8.7964, 8.7661)


{'rnn_type': 'gru', 'n_neurons': 64, 'n_hidden_layers': 1, 'input_batch_normalization': True, 'normalize_signal': True}
Number of parameters: 37985
Got score: 8.7608 (8.7772, 8.7444)
Got score: 8.5115 (8.4480, 8.5749)
Got score: 8.5985 (8.6260, 8.5710)
Got score: 8.4118 (8.4328, 8.3909)
Got score: 8.7220 (8.7736, 8.6705)


{'rnn_type': 'lstm', 'n_neurons': 16, 'n_hidden_layers': 0, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 1265
Got score: 9.1053 (9.1024, 9.1082)
Got score: 8.8140 (8.8241, 8.8040)
Got score: 8.6893 (8.7782, 8.6003)
Got score: 8.8645 (8.8752, 8.8537)
Got score: 9.1155 (9.1227, 9.1082)


{'rnn_type': 'gru', 'n_neurons': 16, 'n_hidden_layers': 0, 'input_batch_normalization': True, 'normalize_signal': True}
Number of parameters: 1025
Got score: 8.9683 (8.9788, 8.9579)
Got score: 8.6729 (8.6858, 8.6599)
Got score: 8.8380 (8.8121, 8.8640)
Got score: 8.7279 (8.6797, 8.7760)
Got score: 8.8727 (8.8479, 8.8974)


In [11]:
mean_scores = [f"{np.mean(scores):0.2f}" for scores in model_scores.values()]
std_scores = [f"{np.std(scores):0.2f}" for scores in model_scores.values()]
n_params = [count_params(model_builder(hyperparameters)) for hyperparameters in
            bayesian_tuner.get_best_hyperparameters(TOP_N)]

df = pd.DataFrame({'mean': mean_scores, 'std': std_scores, 'n_params': n_params}, index=model_scores.keys())
df.index.name = 'Model'
df

,mean,std,n_params
Model,,,
0,8.58,0.15,3377
1,8.60,0.13,37985
2,8.92,0.17,1265
3,8.82,0.11,1025
